In [1]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"


cache_dir = "/root/autodl-tmp"


In [2]:

import transformers
from datasets import Dataset
from transformers import AutoTokenizer,  AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer


In [3]:

ds = Dataset.load_from_disk("./alpaca_data_zh/")
ds

ds[:1]





{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。'],
 'input': [''],
 'instruction': ['保持健康的三个提示。']}

In [4]:


tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh", cache_dir=os.path.join(cache_dir, "bloom-1b4-zh"))
tokenizer


BloomTokenizerFast(name_or_path='Langboat/bloom-1b4-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:

def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }



In [6]:

tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds



Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [7]:

print(tokenizer.decode(tokenized_ds[1]["input_ids"]))
print(tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"]))))





Human: 解释为什么以下分数等同于1/4
输入：4/16

Assistant: 4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>
4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>


In [8]:


model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True, cache_dir = os.path.join(cache_dir, "bloom-1b4-zh"))



In [17]:

# 实现Prompt Tuning fine tuning

from peft import TaskType, get_peft_model, PromptEncoderConfig, PromptEncoderReparameterizationType

"""
prompt tuning时的设置
"""
# # soft prompt
# config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM,num_virtual_tokens=10)

# # hard prompt

# config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM,
#                            prompt_tuning_init=PromptTuningInit.TEXT,
#                            prompt_tuning_init_text="下面是一段人与机器人的对话。",
#                            num_virtual_tokens=len(tokenizer("下面是一段人与机器人的对话。")["input_ids"]),
#                            tokenizer_name_or_path="Langboat/bloom-1b4-zh")


"""
p-tuning使用时的设置
"""
# 默认是MLP进行编码（3层）
config = PromptEncoderConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10) 
# 与上述的结果雷同（3层MLP）
config = PromptEncoderConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10,
                            encoder_reparameterization_type=PromptEncoderReparameterizationType.MLP)

# 定义prompt encoder器为lstm，2层lstm  + 3层MLP
# config = PromptEncoderConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10,
#                             encoder_reparameterization_type=PromptEncoderReparameterizationType.LSTM,
#                             encoder_num_layers=2,
#                             encoder_dropout=0.1,
#                             encoder_hidden_size=None)

config






PromptEncoderConfig(peft_type=<PeftType.P_TUNING: 'P_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=10, token_dim=None, num_transformer_submodules=None, num_attention_heads=None, num_layers=None, encoder_reparameterization_type=<PromptEncoderReparameterizationType.MLP: 'MLP'>, encoder_hidden_size=None, encoder_num_layers=2, encoder_dropout=0.0)

In [18]:
# 创建Prompt Tuning model的模型

model = get_peft_model(model=model,peft_config=config,
                       adapter_name="default", mixed=False)  # 这两个是默认参数


model


PeftModelForCausalLM(
  (base_model): PeftModelForCausalLM(
    (base_model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(46145, 2048)
        (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-23): 24 x BloomBlock(
            (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
              (dense): Linear(in_features=2048, out_features=2048, bias=True)
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (mlp): BloomMLP(
              (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
              (gelu_impl): BloomGelu()
              (dense_4h_to_h): Linear(in_fe

In [19]:

model.print_trainable_parameters()




trainable params: 12,609,536 || all params: 1,315,721,216 || trainable%: 0.9584


In [21]:


args = TrainingArguments(output_dir="/root/autodl-tmp/p_tuning_bloom_1b4",
                        per_device_train_batch_size=8,
                         gradient_accumulation_steps=2,
                        learning_rate=8e-5,
                        logging_steps=10,
                        num_train_epochs=1)

trainer = Trainer(model=model,args = args,train_dataset=tokenized_ds, data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True))

trainer.train()



Step,Training Loss
10,2.344200
20,2.398700
30,2.376000
40,2.323800
50,2.277100
60,2.323500
70,2.336600
80,2.360400
90,2.385700
100,2.337300


TrainOutput(global_step=1679, training_loss=2.3018611512630023, metrics={'train_runtime': 861.518, 'train_samples_per_second': 31.175, 'train_steps_per_second': 1.949, 'total_flos': 2.673169270505472e+16, 'train_loss': 2.3018611512630023, 'epoch': 1.0})

In [24]:

model.save_pretrained(save_directory="/root/autodl-tmp/p_tuning_bloom_1b4/save_pretrained",
                      safe_serialization=False)  # 默认是False





In [25]:


# 测试模型加载部分




from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig


origin_model =  AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True, cache_dir = os.path.join(cache_dir, "bloom-1b4-zh"))

peft_model = PeftModel.from_pretrained(model=origin_model,
                                       model_id="/root/autodl-tmp/p_tuning_bloom_1b4/save_pretrained/",
                                       is_trainable=False,  # 默认是推理模式
                                      )








In [30]:
peft_model = peft_model.cuda()
peft_model



PeftModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(46145, 2048)
      (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0-23): 24 x BloomBlock(
          (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
            (dense): Linear(in_features=2048, out_features=2048, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
          )
        )
      )

In [35]:


ipt = tokenizer("Human: {}\n{}".format("考试有哪些好的技巧", "").strip() + "\n\nAssistant: ", return_tensors='pt').to(peft_model.device)

print(tokenizer.decode(peft_model.generate(**ipt, max_length=256, do_sample=False, )[0], 
                 skip_special_tokens=True))

Human: 考试有哪些好的技巧

Assistant: 考试有很多技巧，但最重要的技巧是保持良好的心态。考试时，考生需要保持良好的心态，保持积极的心态，保持良好的学习状态，这样才能取得好成绩。考试时，考生需要保持良好的心态，保持积极的心态，保持良好的学习状态，这样才能取得好成绩。


In [37]:
model.generate??

Signature: model.generate(*args, **kwargs)
Docstring: <no docstring>
Source:   
    def generate(self, *args, **kwargs):
        peft_config = self.active_peft_config
        self.base_model.prepare_inputs_for_generation = self.prepare_inputs_for_generation
        if hasattr(self.base_model, "model"):
            self.base_model.model.generation_config = self.generation_config
        else:
            self.base_model.generation_config = self.generation_config
        try:
            if not peft_config.is_prompt_learning:
                with self._enable_peft_forward_hooks(*args, **kwargs):
                    kwargs = {k: v for k, v in kwargs.items() if k not in self.special_peft_forward_args}
                    outputs = self.base_model.generate(*args, **kwargs)
            else:
                outputs = self.base_model.generate(**kwargs)
        except:
            self.base_model.prepare_inputs_for_generation = self.base_model_prepare_inputs_for_generation
            raise
  

In [38]:
from transformers.generation import GenerationConfig


# 具体的generate的各个参数控制，查看GenerationConfig的__init__函数

GenerationConfig??



"""
类用于保存生成任务的配置。调用generate支持以下对于text-decoder、text-to-text、speech-to-text和vision-to-text模型的生成方法：

如果num_beams=1且do_sample=False，则使用贪婪搜索，调用~generation.GenerationMixin.greedy_search。
如果penalty_alpha>0且top_k>1，则使用对比搜索，调用~generation.GenerationMixin.contrastive_search。
如果num_beams=1且do_sample=True，则使用多概率采样，调用~generation.GenerationMixin.sample。
如果num_beams>1且do_sample=False，则使用beam搜索，调用~generation.GenerationMixin.beam_search。
如果num_beams>1且do_sample=True，则使用beam搜索多概率采样，调用~generation.GenerationMixin.beam_sample。
如果num_beams>1且num_beam_groups>1，则使用分群束搜索，调用~generation.GenerationMixin.group_beam_search。
如果num_beams>1且constraints!=None或force_words_ids!=None，则使用约束束搜索，调用~generation.GenerationMixin.constrained_beam_search。

在使用这个模型进行文本生成时，您也可以不直接调用上述方法。而是将自定义参数值传递给'generate'方法。

参数说明：

    max_length：控制生成输出的长度，默认为 20。它的值对应于输入提示的长度加上max_new_tokens。如果同时设置了max_new_tokens，则它的效果将被覆盖。
    max_new_tokens：控制要生成的令牌数量，忽略提示中的令牌数量。它的值默认为 0。
    min_length：控制生成序列的最小长度，默认为 0。它的值对应于输入提示的长度加上min_new_tokens。如果同时设置了min_new_tokens，则它的效果将被覆盖。
    min_new_tokens：控制要生成的令牌数量，忽略提示中的令牌数量。它的值默认为 0。
    early_stopping：控制基于 beam 的方法（如 beam-search）的停止条件。它接受以下值：True，表示生成在有num_beams个完整候选项时停止；False，表示应用启发式方法，在找到更好候选项的可能性很小时停止；"never"，表示 beam 搜索过程仅在无法找到更好候选项时停止（经典 beam 搜索算法）。
    max_time：允许计算运行的最大时间，单位为秒。如果分配的时间已过，生成过程仍会完成当前迭代。

这个注释是用于控制生成策略的参数。它包含了以下几个参数：

do_sample（可选，默认为False）：是否使用采样；否则使用贪婪解码。
num_beams（可选，默认为1）：束搜索的束数。1表示不使用束搜索。
num_beam_groups（可选，默认为1）：将num_beams分成若干组，以确保不同束组的多样性。更多详细信息请参考这篇论文(This Paper)。
penalty_alpha（可选）：在对比搜索解码中，平衡模型置信度和退化惩罚的值。
use_cache（可选，默认为True）：模型是否应使用过去的最后一个键/值注意力（如果适用于模型）来加速解码。
"""



Init signature: GenerationConfig(**kwargs)
Source:        
class GenerationConfig(PushToHubMixin):
    # no-format
    r"""
    Class that holds a configuration for a generation task. A `generate` call supports the following generation methods
    for text-decoder, text-to-text, speech-to-text, and vision-to-text models:

        - *greedy decoding* if `num_beams=1` and `do_sample=False`
        - *contrastive search* if `penalty_alpha>0.` and `top_k>1`
        - *multinomial sampling* if `num_beams=1` and `do_sample=True`
        - *beam-search decoding* if `num_beams>1` and `do_sample=False`
        - *beam-search multinomial sampling* if `num_beams>1` and `do_sample=True`
        - *diverse beam-search decoding* if `num_beams>1` and `num_beam_groups>1`
        - *constrained beam-search decoding* if `constraints!=None` or `force_words_ids!=None`
        - *assisted decoding* if `assistant_model` or `prompt_lookup_num_tokens` is passed to `.generate()`

    To learn more about decodi